In [44]:
import pandas as pd

df = pd.read_csv('/home/yjtech2/Desktop/yurim/LLM/DATA/냄새_악취_포함.csv', encoding = 'cp949')
df.head()

,제목,질문내용
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다.,순창군 순창읍 남계리ㅇㅇㅇ일원\n악취 나는 물 이 계속 흘러 내려 물을 오염시키고 ...
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요.,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...
2,노후된 모정 보수 요청ㅓ,전라북도순창군구림면금천리89-2\n치천마을의치천옆에모정이있는데설치한지30년이넘어서나...
3,인도에 패기물 컨테이너 적치로 통행 불편,순창군 순창읍 ㅇㅇ리 ㅇㅇㅇ\n인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 ...
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,첨부 한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유...


## 불필요한 단어 삭제 및 수정

In [ ]:
import re

def text_clean(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    
    # pattern = '[^\w\s]'         # 특수기호제거
    # text = re.sub(pattern, '', text)
    
    pattern = '(?<!\d)-|\b(?!\d+-\d)\d+\b|[^\w\s-]' # 번지수
    text = re.sub(pattern, '', text)
    
    pattern = '\s{2, }'           # 중복 공백 제거
    text = re.sub(pattern, '', text).strip() 
    
    pattern = '\n'            # 줄 바꿈 제거
    text = re.sub(pattern, '', text) 

    # pattern = '\d+'        # 숫자 제거
    # text = re.sub(pattern, '', text) 

    return text 

In [46]:
df['제목'] = df['제목'].apply(text_clean)
df['질문내용'] = df['질문내용'].apply(text_clean)

df

,제목,질문내용
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다,순창군 순창읍 남계리일원 악취 나는 물 이 계속 흘러 내려 물을 오염시키고 있어요
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요,전라북도 순창군 면 마을 꼴목길에 퇴비를 가져다 놓음 바람이불면 골목안쪽으로 냄새가...
2,노후된 모정 보수 요청,전라북도순창군구림면금천리892 치천마을의치천옆에모정이있는데설치한지30년이넘어서나무가...
3,인도에 패기물 컨테이너 적치로 통행 불편,순창군 순창읍 리 인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다 조...
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,첨부 한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유츌되어 ...
...,...,...
111,복합악취의 수인한도는,복합악취의 수인한도는
112,악취배출량 단위 및 산정방법은,악취배출량 단위 및 산정방법
113,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁조정대상인지,단독 또는 공동주택에서 애완견이나 고양이 등 동물사육으로 인하여 소음 악취 등을 유...
114,악취배출사업장 기술지원 및 공공환경시설 악취기술진단 악취물질 측정분석 관련 질의응답,Q1 악취배출사업장 기술지원 악취저감기술 지원의 신청방법과 무상지원 범위는 어느 정...


## 띄어쓰기 및 맞춤법 교정

tensorflow 2.16.2 이상의 버전이 필요한데 pip는 못 함 git에서 tensorflow를 불러와서 설치해야 함

=> 일단 코랩에서 진행(성능 확인을 위한) 

=> 띄어쓰기가 없는 경우는 가눙

=> 잘 못 된 띄어쓰기를 교정하지는 못 함

#### 띄어쓰기

In [ ]:
import json
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import pandas as pd
import re
from difflib import SequenceMatcher
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
def load_data(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'annotation' in item:
            annotation = item['annotation']
            
            if 'err_sentence' in annotation and 'cor_sentence' in annotation:
                data.append({
                    'err_sentence': annotation['err_sentence'],
                    'cor_sentence': annotation['cor_sentence']
                })
                
    df = pd.DataFrame(data)
    return df


# 모델 정의
tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 레이블 정의
label = ["UNK", "PAD", "O", "B", "I", "E", "S"]

def apply_spacing_correction(text):
    if not isinstance(text, str):
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 512 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 512 토큰 단위로 나누기
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            token_list = [tokenizer.cls_token_id]
            for char in chunk:
                token_list.extend(tokenizer.encode(char)[1:-1])  # CLS, SEP 토큰 제외
            token_list.append(tokenizer.sep_token_id)
            
            tkd = torch.tensor(token_list).unsqueeze(0).to(device)
            
            # 모델 예측
            with torch.no_grad():
                outputs = model(tkd)
                pred_idx = torch.argmax(outputs.logits, dim=-1)
            
            # 예측 결과를 문자열로 변환
            pred_sent = ""
            for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
                if char_idx >= len(chunk):  # 인덱스 범위 체크
                    break
                
                curr_label = label[spc_idx]
                if curr_label in ["E", "S"]:  # E나 S 태그가 있으면 띄어쓰기 추가
                    pred_sent += chunk[char_idx] + " "
                else:
                    pred_sent += chunk[char_idx]
            
            corrected_sentences.append(pred_sent.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip()
    
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return text  # 에러 발생시 원본 텍스트 반환

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 띄어쓰기 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(apply_spacing_correction)
    
    return df


# 결과 확인
def add_comparison_columns(df):
    
    def calculate_spacing_similarity(row):
        """
        띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
        """
        if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
            return 0
        
        # 문장의 띄어쓰기를 유지한 채로 비교
        res = row['res_sentence']
        cor = row['cor_sentence']
        
        # 완전히 같으면 1 반환
        if res == cor:
            return 1.0
        
        # 다르면 띄어쓰기를 포함한 유사도 계산
        similarity = SequenceMatcher(None, res, cor).ratio()
        
        # 띄어쓰기 패턴의 정확도를 별도로 계산
        res_spaces = [i for i, char in enumerate(res) if char == ' ']
        cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
        
        # 공통된 띄어쓰기 위치 수 계산
        common_spaces = len(set(res_spaces) & set(cor_spaces))
        total_spaces = len(set(res_spaces) | set(cor_spaces))
        
        # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
        spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
        
        # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
        final_ratio = (similarity + spacing_accuracy) / 2
        
        return round(final_ratio, 3)
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

# 수행
if __name__ == "__main__":
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Start ======')
    df = load_data('/home/yjtech2/Desktop/yurim/LLM/DATA/띄어쓰기문장부호오류.json')
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    print(f'[{_now_time}] ====== Data Preprocessing Start ======')
    processed_df = process_dataframe(df, "err_sentence")
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Preprocessing Finished ======')

    processed_df = processed_df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    def text_clean(text):
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-13 17:05:44.503679: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 17:05:44.529855: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 17:05:44.875217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

처리 완료!
비교 완료
정확도:  0.7749166666666667
비율 평균:  0.916460055555547


#### 띄어쓰기 및 맞춤법 교정

In [3]:
import json
import torch
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    AutoModelForTokenClassification, 
    AutoTokenizer
)
from tqdm import tqdm
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

class TextCorrector:
    def __init__(self):
        # 띄어쓰기 모델 초기화
        self.spacing_tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
        self.spacing_model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")
        
        # 맞춤법 모델 초기화
        self.spelling_tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
        self.spelling_model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
        
        # GPU 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.spacing_model = self.spacing_model.to(self.device)
        self.spelling_model = self.spelling_model.to(self.device)
        
        # 띄어쓰기 레이블
        self.spacing_labels = ["UNK", "PAD", "O", "B", "I", "E", "S"]

    # 띄어쓰기 교정
    def correct_spacing(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 512 - 2
            org_text = text.replace(" ", "")
            results = []
            
            for start_idx in range(0, len(org_text), max_length):
                chunk = org_text[start_idx:start_idx + max_length]
                
                # 토큰화
                tokens = [self.spacing_tokenizer.cls_token_id]
                tokens.extend(sum([self.spacing_tokenizer.encode(char)[1:-1] for char in chunk], []))
                tokens.append(self.spacing_tokenizer.sep_token_id)
                
                # 예측
                with torch.no_grad():
                    inputs = torch.tensor([tokens]).to(self.device)
                    outputs = self.spacing_model(inputs)
                    predictions = torch.argmax(outputs.logits, dim=-1)
                
                # 결과 생성
                result = ""
                for idx, pred in enumerate(predictions.squeeze()[1:-1]):
                    if idx >= len(chunk):
                        break
                    result += chunk[idx]
                    if self.spacing_labels[pred] in ["E", "S"]:
                        result += " "
                        
                results.append(result.strip())
            
            return " ".join(results).strip()
            
        except Exception as e:
            print(f"Spacing error: {str(e)}")
            return text

    # 맞춤법 교정
    def correct_spelling(self, text):
        if not isinstance(text, str) or not text.strip():
            return ""
            
        try:
            max_length = 128 - 2
            chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
            results = []
            
            for chunk in chunks:
                inputs = self.spelling_tokenizer(
                    "맞춤법을 고쳐주세요: " + chunk,
                    return_tensors="pt",
                    truncation=True,
                    max_length=max_length
                ).to(self.device)
                
                outputs = self.spelling_model.generate(
                    **inputs,
                    max_length=max_length,
                    num_beams=5,
                    early_stopping=True
                )
                
                result = self.spelling_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                if result:
                    results.append(result)
            
            return " ".join(results).strip() or ""
            
        except Exception as e:
            print(f"Spelling error: {str(e)}")
            return ""

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        return re.sub('[^\w\s]', '', text)

def load_data(file_path):
    
    # with open(file_path, 'r') as f:
    #     data = json.load(f)
        
    # records = [
    #     {
    #         'err_sentence': item['annotation']['err_sentence'],
    #         'cor_sentence': item['annotation']['cor_sentence']
    #     }
    #     for item in data['data']
    #     if 'annotation' in item
    #     and 'err_sentence' in item['annotation']
    #     and 'cor_sentence' in item['annotation']
    # ]
    
    # df = pd.DataFrame(records).fillna('')
    # print(df.info())
    # return df
    
    df = pd.read_csv(file_path, encoding = 'cp949')
    
    print(df.info())
    
    return df

def process_dataset(file_path):
    # 데이터 로드 및 전처리기 초기화
    print('데이터 로드')
    df = load_data(file_path)
    
    corrector = TextCorrector()
    
    # 진행바 설정
    tqdm.pandas(desc="Processing corrections")
    
    # 띄어쓰기 교정
    print("Applying spacing correction...")
    df['spacing_res_sentence'] = df['질문내용'].progress_apply(corrector.correct_spacing)
    
    # 맞춤법 교정
    print("Applying spelling correction...")
    df['spelling_res_sentence'] = df['spacing_res_sentence'].progress_apply(corrector.correct_spelling)
    
    # 텍스트 클리닝
    df['spelling_res_sentence'] = df['spelling_res_sentence'].apply(corrector.clean_text)
    # df['cor_sentence'] = df['cor_sentence'].apply(corrector.clean_text)
    return df

if __name__ == "__main__":
    file_path = '/home/yjtech2/Desktop/yurim/LLM/DATA/냄새_악취_포함.csv'
    result_df = process_dataset(file_path)
    
    print("\n=== Results ===")
    result_df

데이터 로드
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      116 non-null    object
 1   질문내용    116 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB
None
Applying spacing correction...


Processing corrections: 100%|██████████| 116/116 [00:00<00:00, 169.90it/s]


Applying spelling correction...


Processing corrections: 100%|██████████| 116/116 [00:52<00:00,  2.22it/s]


=== Results ===


In [12]:
result_df

,제목,질문내용,spacing_res_sentence,spelling_res_sentence
0,악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다.,순창군 순창읍 남계리ㅇㅇㅇ일원\n악취 나는 물 이 계속 흘러 내려 물을 오염시키고 ...,순창군 순창읍 남계리 ㅇㅇㅇ 일원\n악 취나 는물 이계 속흘러내 려물 을오염시키 고...,순창군 순창읍 남계리 ᄋᄋᄋ 일원 악 취나 는 물 이계 속 흘러내 려 물 오염시키고...
1,마을 골 목 길에 퇴비를 두어 악취 발생이 심하니 이동 조치해 주세요.,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음\n바람이불면 골목안쪽으로...,전라북도 순창군 ㅇㅇ면 마을 꼴목길에 퇴비를 가져다 놓음 \n바람 이불 면골 목안쪽...,전라북도 순창군 ᄋᄋ 면 마을 꼴 목길에 퇴비를 가져다 놓음 바람 이불 면골 목 안...
2,노후된 모정 보수 요청ㅓ,전라북도순창군구림면금천리89-2\n치천마을의치천옆에모정이있는데설치한지30년이넘어서나...,전라북도 순창군 구림면 금천리 89-2 \n치천마을 의치 천옆 에모정 이있는 데설치...,전라북도 순창군 구림면 금천리 892 치천 마을 의치 천 옆 에모정이 있는 데 설치...
3,인도에 패기물 컨테이너 적치로 통행 불편,순창군 순창읍 ㅇㅇ리 ㅇㅇㅇ\n인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 ...,순창군 순창읍 ㅇㅇ리 ㅇㅇㅇ \n인도 에폐기 물컨테이너 가있 어통 행불 편 및악취 ...,순창군 순창읍 ᄋᄋ리 ᄋᄋᄋ 인도 쓰레기 물 컨테이너가 있어 어통 행불 편 및 악취...
4,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,첨부 한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유...,첨부한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유츌되어 제...,첨부한 사진과 같이 이웃집에서 보일러 기름으로 추정되는 액체가 다량으로 유츌되어 제...
...,...,...,...,...
111,복합악취의 수인한도는?,복합악취의 수인한도는?,복합악취의 수인 한도는?,복합 악취의 수인 한도는
112,악취배출량 단위 및 산정방법은?,악취배출량 단위 및 산정방법,악취 배출량 단위 및 산정 방법,냄새 배출량 단위 및 산정 방법
113,단독 또는 공동주택에서 애완견 등을 사육하여 소음 등을 유발하는 경우 환경분쟁조정대...,"단독 또는 공동주택에서 애완견이나 고양이 등 동물사육으로 인하여 소음, 악취 등을 ...","단독 또는 공동주택에서 애완견이나 고양이 등 동물 사육으로 인하여 소음, 악취 등을...",단독 또는 공동주택에서 애완견이나 고양이 등 동물 사육으로 인하여 소음 및 냄새 등...
114,"악취배출사업장 기술지원 및 공공환경시설 악취기술진단, 악취물질 측정·분석 관련 질의응답",Q1: ○ (악취배출사업장 기술지원) 악취저감기술 지원의 신청방법과 무상지원 범위는...,Q1:○(악취배출사업장 기술지원) 악취저감기술지원의 신청방법과 무상지원 범위는 어느...,Q1 악취 배출 사업장 기술 지원 냄새 저감 기술 지원의 신청 방법과 무상 지원 범...


In [14]:
for i in result_df.iloc[0]:
    print(i, '\n')

악취 나는 뮬이 계속 흘러내려 물 을 오염시키고 있습니다. 

순창군 순창읍 남계리ㅇㅇㅇ일원
악취 나는 물 이 계속 흘러 내려 물을 오염시키고 있어요.ㅠ 

순창군 순창읍 남계리 ㅇㅇㅇ 일원
악 취나 는물 이계 속흘러내 려물 을오염시키 고있어요. 

순창군 순창읍 남계리 ᄋᄋᄋ 일원 악 취나 는 물 이계 속 흘러내 려 물 오염시키고 있어요 



## 불용어 제거

한글에서 불용어를 제거하는 방법은 토큰화 후에 조사, 접속사 등을 제거하는 방법이 가장 간단하다

In [ ]:
import nltk
nltk.download('stopwords')

with open("stop_words.txt", "r", encoding="utf-8") as file:
    stop_words_loaded = eval(file.read())
    
# 결과 텍스트를 단어 리스트로 변환
text_lst = result_df.split()

# 불용어를 제외한 새로운 리스트 생성
text_lst = [word for word in text_lst if word not in stop_words_loaded]

print(text_lst)  # 불용어가 제거된 리스트 출력